In [25]:

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from distutils.version import StrictVersion
import pytesseract
import shutil
import tensorflow as tf
import io
from object_detection.utils import dataset_util, ops
from PIL import Image
import re

import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import inflect
import uuid
from shapely.geometry import Polygon
from mining_pages_utils.image_ocr_utils import double_to_singlepage, load_page, cut_image, ocr_pre_processing_page, ocr_pre_processing_figure, ocr_post_processing_pageid, cut_image_savetemp, cut_image_figure, ocr_post_processing_figid
from mining_pages_utils.dataframe_utils import get_page_labelmap_as_df, get_figid_labelmap_as_df, extract_page_detections, extract_page_detections_new,unfold_pagedetections, page_detections_toframe, extract_detections_figureidv2,humanreadID
from mining_pages_utils.dataframe_utils import extract_pdfid, filter_best_page_detections, select_pdfpages, choose_pageid, filter_best_vesselprofile_detections, merge_info,  provide_pagelist, provide_pdf_path, get_pubs_and_configs, pdf_to_imagev2, handleduplicate_humanreadID
from mining_pages_utils.json_utils import create_find_JSONL, create_constructivisttype_JSONL, create_normativtype_JSONL, create_drawing_JSONL, create_catalog_JSONL, create_trench_JSONL
from mining_pages_utils.tensorflow_utils import create_tf_example_new, create_tf_figid, run_inference_for_page_series, run_inference_for_figure_series, build_detectfn, Df2TFrecord, split
from mining_pages_utils.request_utils import getZenonInfo

if StrictVersion(tf.version.VERSION) < StrictVersion('2.3.0'):
    raise ImportError(
        'Please upgrade your TensorFlow installation to v2.3.* or later!')

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

INPUTDIRECTORY = '/home/images/apply' 
GRAPH = '/frozen_inference_graph.pb'
LABELS = '/label_map.pbtxt'
PAGE_MODEL = '/home/models/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8_miningpagesv12'
SAVEDMODEL = '/saved_model'
FIGID_MODEL = '/home/models/faster_rcnn_resnet101_v1_1024x1024_coco17_miningfiguresv4'
SEG_MODEL = '/home/models/shape_segmentation/train_colab_20200610.h5'
OUTPATH = '/home/images/OUTPUT/'
VESSELLPATH = OUTPATH + 'vesselprofiles/'
SEGMENTPATH = OUTPATH + 'segmented_profiles/'
CSVOUT = OUTPATH + 'mining_pages_allinfo.csv'
CLEANCSVOUT = OUTPATH + 'mining_pages_clean.csv'









In [2]:
publist = get_pubs_and_configs(INPUTDIRECTORY)
publist = publist.apply(getZenonInfo, axis=1)
pdflist = provide_pdf_path(publist)
pdflistv2 = pdflist.apply(pdf_to_imagev2, axis=1)
print('Finished pdf2images')
pagelist = provide_pagelist(pdflistv2)
print('Created primary pagelist')
pagelist = pagelist.apply(extract_pdfid, axis=1)
pagelist = double_to_singlepage(pagelist)
pagelist = provide_pagelist(pdflist)
pagelist = pagelist.apply(extract_pdfid, axis=1)
pagelist = select_pdfpages(pagelist)





For pub_key: ZenonID found pub_info!
For pub_key: ZenonID found pub_info!
['Dicocer_dictionnaire_des_ceramiques_anti.pdf0006-022.png', 'Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png', 'Dicocer_dictionnaire_des_ceramiques_anti.pdf0001-017.png', 'Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png', 'Dicocer_dictionnaire_des_ceramiques_anti.pdf0003-019.png', 'Dicocer_dictionnaire_des_ceramiques_anti.pdf0005-021.png']
17
18
19
20
21
['ock_all.pdf0005-21.png', 'ock_all.pdf0001-6.png', 'ock_all.pdf0002-12.png', 'ock_all.pdf0004-9.png', 'ock_all.pdf0002-11.png', 'ock_all.pdf0003-16.png', 'ock_all.pdf0005-24.png', 'ock_all.pdf0005-23.png', 'ock_all.pdf0001-4.png', 'ock_all.pdf0002-9.png', 'ock_all.pdf0004-10.png', 'ock_all.pdf0002-10.png', 'ock_all.pdf0004-19.png', 'ock_all.pdf0001-8.png', 'ock_all.pdf0004-20.png', 'ock_all.pdf0001-3.png', 'ock_all.pdf0004-17.png', 'ock_all.pdf0005-22.png', 'ock_all.pdf0001-7.png', 'ock_all.pdf0002-6.png', 'ock_all.pdf0003-15.png', 'ock_all.pd

In [3]:

for path in [VESSELLPATH, SEGMENTPATH]:
    if not os.path.exists(path):
        os.makedirs(path)







all_detections_step1 = pd.DataFrame()
miningpagesdetectfn = build_detectfn(PAGE_MODEL + SAVEDMODEL)
for index, row in pagelist.iterrows():
    print('Page ' + os.path.basename(row['page_path']))

    row, page_imgnp = load_page(row)
    input_tensor = tf.convert_to_tensor(page_imgnp)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = miningpagesdetectfn(input_tensor)
    del page_imgnp
    del input_tensor
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    row['page_detections']= detections
    all_detections_step1 = all_detections_step1.append(row)










Loading model...Done! Took 7.267009019851685 seconds
Page Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
/home/images/apply/ZenonID_000891693/Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
Page Dicocer_dictionnaire_des_ceramiques_anti.pdf0001-017.png
/home/images/apply/ZenonID_000891693/Dicocer_dictionnaire_des_ceramiques_anti.pdf0001-017.png
Page Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png
/home/images/apply/ZenonID_000891693/Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png
Page Dicocer_dictionnaire_des_ceramiques_anti.pdf0003-019.png
/home/images/apply/ZenonID_000891693/Dicocer_dictionnaire_des_ceramiques_anti.pdf0003-019.png
Page Dicocer_dictionnaire_des_ceramiques_anti.pdf0005-021.png
/home/images/apply/ZenonID_000891693/Dicocer_dictionnaire_des_ceramiques_anti.pdf0005-021.png
Page ock_all.pdf0001-4.png
/home/images/apply/ZenonID_000066595/ock_all.pdf0001-4.png
Page ock_all.pdf0001-3.png
/home/images/apply/ZenonID_000066595/ock_all.pdf0001-3

In [4]:
#all_detections_step2, keylist =unfold_pagedetections(all_detections_step1)
#all_detections_step22 = extract_page_detections(all_detections_step2, keylist, category_index=get_page_labelmap_as_df(PAGE_MODEL + LABELS))
all_detections_step2 = page_detections_toframe(all_detections_step1).drop(columns='page_detections')

page_category_index = get_page_labelmap_as_df(PAGE_MODEL + LABELS)
all_detections_step2 = all_detections_step2.merge(page_category_index, on=['detection_classes'], how='left')

In [5]:
pageids = filter_best_page_detections(all_detections_step2 , lowest_score=0.8)
bestpages = choose_pageid(pageids)
pageid_raw = pd.DataFrame()

#perform ocr page number
for index, row in bestpages.iterrows():
    img = cut_image(row)
    img2 = ocr_pre_processing_page(img)
    result = pytesseract.image_to_string(img2, config=row['pageid_config'])
    row['newinfo'] = result
    
    pageid_raw = pageid_raw.append(row)
all_detections_step3 = merge_info(all_detections_step2, pageid_raw)
all_detections_step3 = all_detections_step3.apply(ocr_post_processing_pageid, axis=1)

figures = filter_best_vesselprofile_detections(all_detections_step3, lowest_score= 0.8)

['pageid' 'pageinfo']


In [6]:
for index, row in figures.iterrows():
    print(row['detection_scores'])

0.9998767375946045
0.9998742341995239
0.9998687505722046
0.9997594952583313
0.9997525811195374
0.9996956586837769
0.999976634979248
0.9999723434448242
0.999958872795105
0.9999521970748901
0.9999210834503174
0.9998984336853027
0.9998940229415894
0.9998748302459717
0.9998428821563721
0.9998400211334229
0.9998093247413635
0.9997946619987488
0.9997944235801697
0.9997907280921936
0.9997782111167908
0.99974125623703
0.9996761083602905
0.9993948936462402
0.9984133243560791
0.9999929666519165
0.9999884366989136
0.9999696016311646
0.9999227523803711
0.9998990297317505
0.9998562335968018
0.9997602105140686
0.9997115731239319
0.9995508790016174
0.9994698166847229
0.9994298815727234
0.9992451667785645
0.9990218877792358
0.9999921321868896
0.9999879598617554
0.9999769926071167
0.9999449253082275
0.9999217987060547
0.9999076128005981
0.9998973608016968
0.9998469352722168
0.9998447895050049
0.9998188614845276
0.9998015761375427
0.9997215867042542
0.9999973773956299
0.9999967813491821
0.99999439716339

In [7]:
#detect figure id


figures_step1 = pd.DataFrame()
figure_category_index = get_figid_labelmap_as_df(FIGID_MODEL + LABELS)
miningfiguresdetectfn = build_detectfn(FIGID_MODEL + SAVEDMODEL)

for index, row in figures.iterrows():
    print('Figure from ' + os.path.basename(row['page_path']))
    row, figure_imgnp = cut_image_savetemp(row, VESSELLPATH)
    input_tensor = tf.convert_to_tensor(figure_imgnp)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = miningfiguresdetectfn(input_tensor)
    del figure_imgnp
    del input_tensor
    num_detections = int(detections.pop('num_detections'))
    #print(type(detections['detection_multiclass_scores']))
    detectionsDL = { key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detectionsDL['detection_classesname'] = [ figure_category_index.loc[figure_category_index['figid_detection_classes'] == i, 'figid_detection_classesname'].item() for i in detectionsDL['detection_classes'] ]
    detectionsLD = [{key : value[i] for key, value in detectionsDL.items()} 
         for i in range(num_detections)] 
    #print(detectionsLD)
    row['figure_detections'] = detectionsLD
    print(row['figure_detections'][0]['detection_scores'])
    row['figure_num_detections'] = num_detections       
    figures_step1 = figures_step1.append(row)

#figid_category_index = get_figid_labelmap_as_df(PAGE_MODEL + LABELS)
#figures_step2 = 
#figure_detections_names = [detection['detection_classes'] :  for detection in figures_step1['figure_detections']]

#figures_step1['figure_detections'].merge(
#figid_category_index, on=['figid_detection_classes'], how='left')






Loading model...Done! Took 12.532712459564209 seconds
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
0.014240802
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
0.9998679
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
0.018021513
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
0.9999523
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
0.0082072355
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0002-018.png
0.9999182
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png
0.0030345882
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png
0.010279994
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png
0.0024342234
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png
0.002622787
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0004-020.png
0.0018512765
Figure from Dicocer_dictionnaire_des_ceramiques_anti.pdf0

In [8]:

def filterFigureDetections(series):
    if series['frame_contains']:
        if series['detection_classesname'] in series['allowedFrameDetections']:
            detections = [i for i in series['figure_detections'] if i['detection_classesname'] in series['frame_contains']]
            #print(detections)
            detections_df = pd.DataFrame(detections)
            detections_df = detections_df[detections_df['detection_scores'] == detections_df.groupby('detection_classesname')['detection_scores'].transform('max')]
            series['bestfigure_detections'] = detections_df.to_dict('records')
    if series['figure_contains'] :
        if series['detection_classesname'] in series['allowedFigureDetections'] :
            detections = [i for i in series['figure_detections'] if i['detection_classesname'] in series['figure_contains']]
            detections_df = pd.DataFrame(detections)
            detections_df = detections_df[detections_df['detection_scores'] == detections_df.groupby('detection_classesname')['detection_scores'].transform('max')]
            series['bestfigure_detections'] = detections_df.to_dict('records')
    return series


figures_step2 = figures_step1.apply(filterFigureDetections, axis=1)
for index, row in figures_step2.iterrows():
    print(row['figure_path'])
    if type(row['bestfigure_detections']) == list:
        for i in row['bestfigure_detections']:
            print(i['detection_classesname'], i['detection_scores'])




/home/images/OUTPUT/vesselprofiles/bd573530-968b-4463-bd51-eacc16e9b980.png
/home/images/OUTPUT/vesselprofiles/c5502588-17cc-457a-bab0-5e20db5e9f9d.png
textblock 0.9998679161071777
infoframeid 0.05993630364537239
/home/images/OUTPUT/vesselprofiles/b06c3fc5-f522-4b51-93c3-4a51af633191.png
/home/images/OUTPUT/vesselprofiles/e5410cc5-37f8-4236-8a67-7d408525ca93.png
textblock 0.9999523162841797
infoframeid 0.003718240186572075
/home/images/OUTPUT/vesselprofiles/bd225c3a-7da7-4b97-b017-bb7b74489bc4.png
/home/images/OUTPUT/vesselprofiles/54ae6132-5175-4ce0-aca3-458cb66af904.png
textblock 0.9999182224273682
infoframeid 0.4431811571121216
/home/images/OUTPUT/vesselprofiles/1b02ad54-ae50-4022-a0b6-321ecfcd14a0.png
/home/images/OUTPUT/vesselprofiles/5e617920-1824-4207-82f8-0ba7bb8a84e8.png
/home/images/OUTPUT/vesselprofiles/d8956908-19d9-473c-a95a-4ff670651ee6.png
/home/images/OUTPUT/vesselprofiles/c4cece21-6f47-4455-b47a-f23a9e8b49c1.png
/home/images/OUTPUT/vesselprofiles/609fac3d-25a1-4fac-9e5

In [9]:
#perform ocr figid

def ocr_post_processing_figure(row, detection):
    if row[str(detection['detection_classesname']) + '_raw']:
        if str(detection['detection_classesname']) + '_exclude_strings' in list(row.keys()):
            for exclude_string in row[str(detection['detection_classesname']) + '_exclude_strings']:
                row[str(detection['detection_classesname']) + '_raw'].replace(exclude_string,"")
        else:
            print(str(detection['detection_classesname']) + '_exclude_strings' + ' not written in config - No strings will be excluded.')
        
        if str(detection['detection_classesname']) + '_regex' in list(row.keys()):
            regex = re.compile(row[str(detection['detection_classesname']) + '_regex'])
            result = re.search(regex, row[str(detection['detection_classesname']) + '_raw'])
            if result:
                row[str(detection['detection_classesname']) + '_clean'] = result.group(1)
            else:
                row['figid_clean'] = 'none'
        else:
            print(str(detection['detection_classesname']) + '_regex' + ' not written in config - No regex filter will be applied')

    return row

figures_step3 = pd.DataFrame()
for index, row in figures_step2.iterrows():
    print('OCR ' + os.path.basename(row['figure_path']))
    detections = []
    if type(row['bestfigure_detections']) == list:
        for detection in row['bestfigure_detections']:
            detection = cut_image_figure(row, detection)
            detection = ocr_pre_processing_figure(detection)
            print(detection['detection_classesname'])
            if str(detection['detection_classesname']) + '_config' in list(row.keys()):
                row[str(detection['detection_classesname']) + '_raw'] = pytesseract.image_to_string(detection['imgnp'], config=row[str(detection['detection_classesname']) + '_config'])
                print(row[str(detection['detection_classesname']) + '_raw'])
            else:
                row[str(detection['detection_classesname']) + '_raw'] = pytesseract.image_to_string(detection['imgnp'], config=row['pageid_config'])
                print(row[str(detection['detection_classesname']) + '_raw'])
        row['figure_detections'] = detections.append(detection)
        row = ocr_post_processing_figure(row, detection)
        del detection['imgnp']
        

    figures_step3 = figures_step3.append(row)





OCR bd573530-968b-4463-bd51-eacc16e9b980.png
OCR c5502588-17cc-457a-bab0-5e20db5e9f9d.png
textblock
amphore

Class. ref. Ostia IV, 257 n°12: type “africano
grande”. Equiv. Keay 1984, types IV-V.
Amphore cylindnque a bord en amande ou en
bandeau arrondi (var. “senza gradino”) parfois
souligne par un leger ressaut exteme (var. “con
gradino’).

Utlisation: transporter, stocker.

infoframeid
reraa nau

OCR b06c3fc5-f522-4b51-93c3-4a51af633191.png
OCR e5410cc5-37f8-4236-8a67-7d408525ca93.png
textblock
amphore

Class. ref. Ostia IV, 256 n°9: “africana piccola”.
Equiv. Keay 1984, type Illa; Panella 1982, type
WV.

Amphore cylindnque a pied court, souvent
creux; col bas a bord en bandeau evase.
Utilisation: transporter, stocker.

Chrono.: 180/380.

infoframeid
A-AFR 1a

OCR bd225c3a-7da7-4b97-b017-bb7b74489bc4.png
OCR 54ae6132-5175-4ce0-aca3-458cb66af904.png
textblock
amphore

Class. ref. Ostia IV, 256 n°10; Panella 1982.
Equiv. Keay 1984, type IIIb.

Amphore cylindrque a fond creux; col b

In [10]:




def humanreadID(Series):
    humanreadID = ''
    if Series['patternHRID']:
        for element in Series['patternHRID']:
            if element in list(Series.keys()):
                humanreadID += str(Series[element]) + '_'
            else:
                print('HRID Element: ' + element + ' does not exist.')
                humanreadID += element + '_'
    else:

        humanreadID = str(Series['figure_tmpid'])

    Series['HRID'] = humanreadID.rstrip('_')
    return Series

figures_step3 = figures_step3.apply(humanreadID, axis=1)
figures_step3 = handleduplicate_humanreadID(figures_step3)

In [11]:

with open(OUTPATH + 'catalogs.jsonl', 'w') as f:
    pubs = figures_step3[['pub_key', 'pub_value']].drop_duplicates()
    pubs.apply(create_catalog_JSONL, file=f, axis=1)
with open(OUTPATH + 'trenches.jsonl', 'w') as f:
    pubs = figures_step3[['pub_key', 'pub_value']].drop_duplicates()
    pubs.apply(create_trench_JSONL, file=f, axis=1)
with open(OUTPATH + 'types.jsonl', 'w') as f:
    figures_step3.apply(create_constructivisttype_JSONL, file=f, axis=1)
with open(OUTPATH + 'types_standalone.jsonl', 'w') as f:
    figures_step3.apply(create_normativtype_JSONL, file=f, axis=1)
with open(OUTPATH + 'finds.jsonl', 'w') as f:
    figures_step3.apply(create_find_JSONL, file=f, axis=1)
with open(OUTPATH + 'drawings.jsonl', 'w') as f:
    figures_step3.apply(create_drawing_JSONL, file=f, axis=1)










#Profile segmentation
#print('Perform image segmentation')
#run_vesselprofile_segmentation(VESSELLPATH, SEGMENTPATH, SEG_MODEL)

KeyError: 'figid_raw'

In [44]:
figures_step3.to_csv(CSVOUT)
figures_clean = figures_step3[['pub_key','pub_value','figure_tmpid','HRID','detection_scores', 'detection_classesname','page_imgname','pageid_raw','figureid_raw','pageid_clean','figureid_clean', 'infoframeid_clean', 'figureinfo_clean', 'textblock_clean','pageinfo_raw','figure_path','page_path']]
figures_clean.to_csv(CLEANCSVOUT)

In [28]:
def create_tf_example_new(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    fileid = os.path.basename(group.filename)
    fileid = fileid.encode('utf8')
    filename = group.filename.encode('utf8')
    image_format = b'png'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        box = row['detection_boxes']
        ymin, xmin, ymax, xmax = box
        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)
        classes_text.append(
            row['detection_classesname'].encode('utf8'))
        classes.append(int(row['detection_classes']))
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(fileid),
        # 'image/source_id': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(fileid),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def create_tf_example_figure(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    fileid = os.path.basename(group.filename)
    fileid = fileid.encode('utf8')
    filename = group.filename.encode('utf8')
    image_format = b'png'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        if type(row['bestfigure_detections']) == list:
            for detection in row['bestfigure_detections']:
                box = detection['detection_boxes']
                ymin, xmin, ymax, xmax = box
                xmins.append(xmin)
                xmaxs.append(xmax)
                ymins.append(ymin)
                ymaxs.append(ymax)
                classes_text.append(
                    detection['detection_classesname'].encode('utf8'))
                classes.append(int(detection['detection_classes']))
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(fileid),
        # 'image/source_id': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(fileid),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def Df2TFrecord(df, imagecolumn, outpath):
    if imagecolumn is 'page_path':
        prefix = ''

    writer = tf.io.TFRecordWriter(outpath)
    grouped = split(df, imagecolumn)
    #print(grouped)

    for group in grouped:
        if imagecolumn is 'figure_path':
        
            tf_example = create_tf_example_figure(group,  outpath)
            writer.write(tf_example.SerializeToString())
        if imagecolumn is 'page_path':
            tf_example = create_tf_example_new(group,  outpath)
            writer.write(tf_example.SerializeToString())

    writer.close()

In [30]:
shutil.copyfile(PAGE_MODEL + LABELS, OUTPATH +'pages_label_map.pbtxt')

mining_pages_detections = figures_step3.append(bestpages)
mining_pages_detections2 = mining_pages_detections.reindex(axis=0)
pages_grouped = mining_pages_detections2.groupby('catalog_id')

for name, group in pages_grouped:
    imgdir = os.path.join(OUTPATH, name + '_pages/')
    if not os.path.exists(imgdir):
        os.makedirs(imgdir)
    Df2TFrecord(group, 'page_path', OUTPATH + name +'_pages.tfrecord')
    for index, row in group.iterrows():
        imgoutpath = imgdir + os.path.basename(row['page_path'])
        #print(imgoutpath)
        if not os.path.exists(imgoutpath):
            shutil.copyfile(row['page_path'], imgoutpath)

shutil.copyfile(FIGID_MODEL + LABELS, OUTPATH + 'figures_label_map.pbtxt')
figures_step3 = figures_step3[figures_step3.detection_boxes.notnull()]

figures_grouped = figures_step3.groupby('catalog_id')

for name, group in figures_grouped:
    imgdir = os.path.join(OUTPATH, name + '_figures/')
    if not os.path.exists(imgdir):
        os.makedirs(imgdir)
    Df2TFrecord(group, 'figure_path', OUTPATH + name +'_figures.tfrecord')
    for index, row in group.iterrows():
        imgoutpath = imgdir + os.path.basename(str(row['figure_path']))
        #print(imgoutpath)
        if not os.path.exists(imgoutpath):
            shutil.copyfile(row['figure_path'], imgoutpath)
